In [1]:
%matplotlib inline
# Import the dependencies.
import pandas as pd
import gmaps
import requests

# Import the API key.
from config import g_key
import matplotlib.pyplot as plt

In [2]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [3]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("Data/WeatherPy_challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description,Rain,Snow
0,0,Jamestown,US,2020-02-24 08:36:54,42.10,-79.24,42.80,51,1,10.29,clear sky,0.00,0.0
1,1,Vao,NC,2020-02-24 08:39:37,-22.67,167.48,79.02,74,35,12.84,scattered clouds,0.00,0.0
2,2,Mbanza-Ngungu,CD,2020-02-24 08:39:37,-5.25,14.87,75.61,89,100,2.35,light rain,0.88,0.0
3,3,Mandalgovi,MN,2020-02-24 08:39:37,45.76,106.27,23.05,58,100,18.63,overcast clouds,0.00,0.0
4,4,Upernavik,GL,2020-02-24 08:39:38,72.79,-56.15,-21.44,92,22,1.28,few clouds,0.00,0.0


In [4]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Date                    object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Weather description     object
Rain                   float64
Snow                   float64
dtype: object

In [5]:
type(city_data_df)

pandas.core.frame.DataFrame

In [6]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [7]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [8]:
# Filter the dataset to find the cities that fit the criteria.I added dropna() at the end of the filter to remove all values with null values
preferred_cities_df = city_data_df[(city_data_df["Country"] == "NZ")]

#I could do it in a separated line
preferred_cities_df = preferred_cities_df.dropna()

preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description,Rain,Snow
35,35,Bluff,NZ,2020-02-24 08:39:44,-46.60,168.33,64.00,62,100,4.00,overcast clouds,0.0,0.0
46,46,Tuatapere,NZ,2020-02-24 08:39:46,-46.13,167.68,64.00,62,100,4.00,overcast clouds,0.0,0.0
63,63,Mataura,NZ,2020-02-24 08:39:50,-46.19,168.86,64.00,62,100,4.00,overcast clouds,0.0,0.0
102,102,Ahipara,NZ,2020-02-24 08:39:59,-35.17,173.17,66.60,75,0,10.74,clear sky,0.0,0.0
124,124,Kaeo,NZ,2020-02-24 08:40:03,-35.10,173.78,67.89,72,0,12.59,clear sky,0.0,0.0
134,134,Waipawa,NZ,2020-02-24 08:40:05,-41.41,175.52,62.60,72,76,4.70,broken clouds,0.0,0.0
235,235,Kaitangata,NZ,2020-02-24 08:40:28,-46.28,169.85,61.00,71,100,8.32,overcast clouds,0.0,0.0
242,242,Opunake,NZ,2020-02-24 08:40:29,-39.46,173.86,63.00,61,38,5.99,scattered clouds,0.0,0.0
285,285,Te Anau,NZ,2020-02-24 08:40:39,-45.42,167.72,53.80,82,70,8.46,broken clouds,0.0,0.0
497,497,Murchison,NZ,2020-02-24 08:41:26,-41.80,172.33,51.30,95,0,1.45,clear sky,0.0,0.0


In [9]:
preferred_cities_df.count()

City_ID                12
City                   12
Country                12
Date                   12
Lat                    12
Lng                    12
Max Temp               12
Humidity               12
Cloudiness             12
Wind Speed             12
Weather description    12
Rain                   12
Snow                   12
dtype: int64

In [10]:
#checking any nan
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather description    0
Rain                   0
Snow                   0
dtype: int64

In [11]:
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Weather description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
35,Bluff,NZ,64.00,overcast clouds,-46.60,168.33,
46,Tuatapere,NZ,64.00,overcast clouds,-46.13,167.68,
63,Mataura,NZ,64.00,overcast clouds,-46.19,168.86,
102,Ahipara,NZ,66.60,clear sky,-35.17,173.17,
124,Kaeo,NZ,67.89,clear sky,-35.10,173.78,
134,Waipawa,NZ,62.60,broken clouds,-41.41,175.52,
235,Kaitangata,NZ,61.00,overcast clouds,-46.28,169.85,
242,Opunake,NZ,63.00,scattered clouds,-39.46,173.86,
285,Te Anau,NZ,53.80,broken clouds,-45.42,167.72,
497,Murchison,NZ,51.30,clear sky,-41.80,172.33,


In [12]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [13]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        
hotel_df.head(10)

,City,Country,Max Temp,Weather description,Lat,Lng,Hotel Name
35,Bluff,NZ,64.00,overcast clouds,-46.60,168.33,Bluff Homestead - Guesthouse & Campervan Park
46,Tuatapere,NZ,64.00,overcast clouds,-46.13,167.68,Ron and Tony's Bed & Breakfast
63,Mataura,NZ,64.00,overcast clouds,-46.19,168.86,Ellie's Villa
102,Ahipara,NZ,66.60,clear sky,-35.17,173.17,Ahipara Holiday Park
124,Kaeo,NZ,67.89,clear sky,-35.10,173.78,Bushido Lodge
134,Waipawa,NZ,62.60,broken clouds,-41.41,175.52,Tora Coastal Walk
235,Kaitangata,NZ,61.00,overcast clouds,-46.28,169.85,Kaitangata Motor Camp
242,Opunake,NZ,63.00,scattered clouds,-39.46,173.86,Walton Cottage
285,Te Anau,NZ,53.80,broken clouds,-45.42,167.72,Kingsgate Hotel Te Anau
497,Murchison,NZ,51.30,clear sky,-41.80,172.33,OFF THE BEATEN TRACK ACCOMMODATION


In [16]:
#picked 4 hotels
Tuatapere = (-46.13,167.68)
Kaitangata = (-46.28,169.85)
Bluff = (-46.60,168.33)
Christchurch = (-43.53,172.63)

In [17]:
#centering the map
direction_x = hotel_df["Lat"].median()
direction_y = hotel_df["Lng"].median()
center_coor = (direction_x,direction_y)
center_coor

(-43.945, 170.85)

In [18]:
# making travel map
locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=center_coor, zoom_level=5.8)
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Weather description}</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

#Hotel info
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#Travel itenary
Tuatapere2Christchurch_via_Bluff_via_Kaitangata = gmaps.directions_layer(Tuatapere, Christchurch, waypoints=[Bluff,Kaitangata],show_markers=False,travel_mode='DRIVING')


fig.add_layer(Tuatapere2Christchurch_via_Bluff_via_Kaitangata)
fig.add_layer(marker_layer)


# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))